In [87]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

import string
import warnings
import missingno
warnings.filterwarnings('ignore')


import pandas as pd
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np
from numpy.random import gamma
from numpy.random import multivariate_normal
from scipy.stats import multivariate_t
from scipy.stats import f
import matplotlib.pyplot as plt



cf.go_offline(connected = True)


import pandas as pd
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np
from numpy.random import gamma
from numpy.random import multivariate_normal
from scipy.stats import multivariate_t
from scipy.stats import f
import matplotlib.pyplot as plt
import missingno


cf.go_offline(connected = True)


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import numpy as np
import pandas as pd
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import string
import warnings
import missingno
import statistics
import math
import scipy

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def make_time_series_3(row):

    i = str(row['공고게시일자'])
    year = i[0:4]
    month = i[4:6]
    date =  i[6:8]

    row['공고게시일자'] = pd.to_datetime(year+month+date,format='%Y%m%d')

    return row

In [3]:
df = pd.read_csv('용역_입찰공고_내역_2020.csv',encoding ='utf-8',error_bad_lines= False)
df = pd.concat([df,pd.read_csv('용역_입찰공고_내역_2019.csv',encoding ='utf-8',error_bad_lines= False)])
df = pd.concat([df,pd.read_csv('용역_입찰공고_내역_2018.csv',encoding ='utf-8',error_bad_lines= False)])
df_용역_입찰공고_내역 = pd.concat([df,pd.read_csv('용역_입찰공고_내역_2017.csv',encoding ='utf-8',error_bad_lines= False)])   

df_용역_입찰공고_내역_2 = df_용역_입찰공고_내역[['등록유형','공고게시일자'
,'조달구분',
'입찰공고차수',
'수요기관명',
'수요기관코드',
'긴급공고여부',
'정정공고여부',
'취소공고여부',
'추정가격',                                 
'배정예산',
'입찰방식',
'재입찰허용여부',
'낙찰자결정방법',
'입찰계약방법',
'예가방법',                                 
'업종제한내용',
'지역제한내용',
 ]].copy()



In [7]:
df_용역_입찰공고_내역_2 = df_용역_입찰공고_내역_2.apply(make_time_series_3,axis=1)


NameError: name 'df_용역_입찰공고_내역_3' is not defined

In [80]:
df_용역_입찰공고_내역_3 = df_용역_입찰공고_내역_2.dropna(subset=['배정예산']).copy()
df_make_s_chart = df_용역_입찰공고_내역_3[['공고게시일자','배정예산','수요기관코드']].copy()

In [227]:
n=20
c4 =math.sqrt( 2/(n-1) ) * ( scipy.special.gamma(n/2)  / scipy.special.gamma((n-1)/2))
1 - (3 / c4)* math.sqrt(1-pow(c4,2))

0.510230589401996

In [231]:
def xs_chart_1(data , constant):
    length = data.shape[0]

    lst_ucl = []
    lst_lcl = []
    lst_all_std = []
    lst_std = []
    
    all_std =  constant
    
 
    for i in range(length):
        
        n = data.loc[i, 'count']
        mean = data.loc[i,'mean']
        std = data.loc[i,'std']

            
        if n <= 300:
            c4 = math.sqrt( 2/(n-1) ) * ( scipy.special.gamma(n/2)  / scipy.special.gamma((n-1)/2))
        else:
            c4 = (4*n - 4) / (4*n - 3)

        b3 = 1 - (3 / c4)* math.sqrt(1-pow(c4,2))
        b4 = 1 + (3 / c4) * math.sqrt(1-pow(c4,2))
        

        
        if b3 < 0:
            b3 = 0

        lst_lcl.append(b3 * all_std)
        lst_ucl.append(b4 * all_std)
        lst_std.append(std)
    
    each_std = lst_std
    

    return each_std, all_std, lst_ucl , lst_lcl

In [232]:
def make_automation(code , time_interval):
    
    
    df_make_s_chart_2_0 = df_make_s_chart.groupby(['수요기관코드',pd.Grouper(key = '공고게시일자', freq = time_interval)]).agg({np.std,'count','mean'})
    
    
    if sum([i[0] == str(code) for i in df_make_s_chart_2_0.index.tolist()]) == 0:
        print('없는 기관이거나 최상위 기관의 코드인지를 확인하십시오.')
        return code
 
    
    
    b= df_make_s_chart_2_0[[i[0] == str(code) for i in df_make_s_chart_2_0.index.tolist()]]['배정예산'].copy()
    
    
    if b['std'].isnull().sum() != 0:
        print('표준편차가 0인 부분이 있습니다.')
        return b

    
    b_1 = b.reset_index()[['mean','std','count']].copy()

    
    
    if b_1[['count']].isnull().sum().values[0] != 0:
        print('NaN error')
        return b_1

    test = []
    for i in range(b_1.shape[0]):
        n = b_1.loc[i, 'count']
        std = b_1.loc[i,'std']
        test.append( (n-1)*pow(std,2) )

    w = np.sqrt( sum(test) / (b_1.apply(sum)['count'] - b_1.shape[0]) ) 


    each_s, s_bar, s_ucl , s_lcl = xs_chart_1(b_1, w)

    df_ucl = pd.DataFrame(s_ucl,columns = ['ucl'])
    df_lcl = pd.DataFrame(s_lcl,columns = ['lcl'])
    
    

    fig = go.Figure()

    fig.add_trace(go.Scatter(x = b_1.index ,y = [s_bar] * b_1.shape[0],line_shape='hv',name= 'Mean'))

    fig.add_trace(go.Scatter(x = b_1.index ,y = each_s,mode= 'markers+lines',name= 'Monitoring Statistics',marker_color = 'gray'))

    fig.add_trace(go.Scatter(x = b_1.index ,y = df_lcl['lcl'],mode= 'markers', marker_symbol='line-ew',
                         marker_line_color="red", marker_color="red",
                           marker_line_width=2, marker_size=15,name= 'LCL'))

    fig.add_trace(go.Scatter(x = b_1.index ,y = df_ucl['ucl'],mode= 'markers',name= 'UCL', marker_symbol='line-ew',
                         marker_line_width=2, marker_size=15,marker_line_color="blue", marker_color="blue"))




    fig.update_layout(width=1000, height=600,
                  yaxis_range=[10e+07,300e+07],              
                 xaxis_range=[0,50],
                  title = "This X_S Control (수요기관코드 : " +code+ ") ")
    fig.show()
    
    df_all = pd.concat([df_ucl,pd.DataFrame(each_s),df_lcl],axis=1)

        
    return b_1,df_all

In [233]:
df_small, df_test = make_automation(str('6110000'),'2W')

In [221]:
df_small['count'].iplot(mode ='markers')

In [234]:
df_test

,ucl,0,lcl
0,8.785949e+08,4.789744e+08,4.276341e+08
1,8.112574e+08,1.658602e+09,4.949717e+08
2,8.063452e+08,2.109541e+08,4.998839e+08
3,7.878406e+08,2.601172e+08,5.183884e+08
4,7.807914e+08,4.028515e+08,5.254377e+08
...,...,...,...
100,8.308653e+08,4.603481e+08,4.753638e+08
101,7.761751e+08,4.745845e+08,5.300539e+08
102,8.027288e+08,3.741552e+08,5.035003e+08
103,9.818673e+08,3.792322e+08,3.243618e+08


In [212]:
df_test

,ucl,0,lcl
0,9.744488e+08,5.898542e+08,3.092822e+08
1,1.264242e+09,5.304786e+08,1.948910e+07
2,1.008885e+09,1.110943e+08,2.748465e+08
3,8.797724e+08,3.070031e+08,4.039586e+08
4,9.407119e+08,3.038275e+09,3.430191e+08
...,...,...,...
287,9.848300e+08,2.162205e+08,2.989011e+08
288,1.648420e+09,1.609333e+08,0.000000e+00
289,1.164928e+09,4.971846e+08,1.188026e+08
290,1.164928e+09,2.074136e+08,1.188026e+08


In [200]:
df_make_s_chart['수요기관코드'].value_counts()

1690000    11160
6110000     6941
1613000     2844
B551505     2368
B552584     2365
           ...  
Z002811        1
ZT02488        1
7679029        1
7631209        1
Z002818        1
Name: 수요기관코드, Length: 17246, dtype: int64